In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"Keras version: {keras.__version__}")

In [ ]:
!pip install kaggle

import os
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

# Download skin cancer dataset from Kaggle
# You can replace this with any skin cancer dataset from Kaggle
dataset_name = "fanconic/skin-cancer-malignant-vs-benign"  # Example dataset
download_path = "./skin_cancer_dataset"

# Create directory if it doesn't exist
os.makedirs(download_path, exist_ok=True)

# Download the dataset
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

print(f"Dataset downloaded to: {download_path}")
print("Available files:")
for root, dirs, files in os.walk(download_path):
    for file in files[:10]:  # Show first 10 files
        print(f"  {os.path.join(root, file)}")
    if len(files) > 10:
        print(f"  ... and {len(files) - 10} more files")
    break

In [ ]:
# Check the downloaded dataset structure
data_dir = "./skin_cancer_dataset"

print(f"Dataset location: {data_dir}")
print("\nDataset structure:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files in each directory
        print(f"{sub_indent}{file}")
    if len(files) > 5:
        print(f"{sub_indent}... and {len(files) - 5} more files")
        
print(f"\nTotal number of directories: {len(dirs) if 'dirs' in locals() else 'N/A'}")
print(f"Ready for CNN training!")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2

# Set up data directories
data_dir = "./skin_cancer_dataset"
img_height, img_width = 224, 224
batch_size = 32

# Create data generators
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Load training data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Build CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

# Train the model
epochs = 100
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose=1
)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# Print final training results
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"\nFinal Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

In [ ]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

# Generate predictions for confusion matrix
validation_generator.reset()
predictions = model.predict(validation_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

In [ ]:
# Save the trained model
model.save('skin_cancer_cnn_model.h5')
print("Model saved as 'skin_cancer_cnn_model.h5'")

# Function to predict on a single image
def predict_image(image_path, model, class_labels):
    """
    Predict the class of a single image
    """
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension
    img_array /= 255.0  # Normalize
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_idx]
    predicted_class = class_labels[predicted_class_idx]
    
    return predicted_class, confidence, predictions[0]

# Function to display prediction results
def display_prediction(image_path, model, class_labels):
    """
    Display image with prediction results
    """
    predicted_class, confidence, all_predictions = predict_image(image_path, model, class_labels)
    
    # Display image
    img = plt.imread(image_path)
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f'Predicted: {predicted_class}\nConfidence: {confidence:.2%}')
    plt.axis('off')
    
    # Display prediction probabilities
    plt.subplot(1, 2, 2)
    plt.bar(class_labels, all_predictions)
    plt.title('Prediction Probabilities')
    plt.ylabel('Probability')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return predicted_class, confidence

print("Prediction functions defined. Use predict_image() or display_prediction() to test on new images.")

In [ ]:
# Example: Test prediction on a sample image from the validation set
# Get a few sample images from validation data
import random

# Get some sample images from the dataset
sample_images = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            sample_images.append(os.path.join(root, file))
            if len(sample_images) >= 5:  # Get 5 sample images
                break
    if len(sample_images) >= 5:
        break

if sample_images:
    print(f"Testing prediction on {len(sample_images)} sample images:")
    print("Class labels:", list(train_generator.class_indices.keys()))
    
    # Test on first sample image
    if len(sample_images) > 0:
        sample_image = sample_images[0]
        print(f"\nTesting on: {sample_image}")
        predicted_class, confidence = display_prediction(
            sample_image, 
            model, 
            list(train_generator.class_indices.keys())
        )
        print(f"Prediction: {predicted_class} ({confidence:.2%} confidence)")
else:
    print("No sample images found. Please ensure the dataset is properly downloaded and extracted.")
    print("You can test predictions later using:")
    print("display_prediction('path_to_your_image.jpg', model, list(train_generator.class_indices.keys()))")